<h1>Coursera Capstone Data Collision</h1>

This notebook is the final capstone of the IBM Data Science Coursera course. With this notebook, the <b>Data Collisions - All years</b> data set for seattle will be evaluated and a prediction will be done for the probability of a collission under some given circumstances.

To achieve this, the data will be analyzed, avaluated, preprocessed and models will be created and tested to find the best model and prediction. 

<h2>Business Problem</h2>

There are several variables who can indicate that there is a high risk to have an accident, variables which influence the risk could be for example the weather condition (are the roads slippy?), time of the day (is the driver tired?) and so on. 
With the following examination we want to have a deeper look onto the weakest participants in traffic, the pedestrians. Pedestrians are often not the root cause of accidents but if they are involved, the often have severe injuries. 
By analyzing the data set we want to find out, under what circumstances the risk of an accident with pedestrians is high and when the injuries are worse. The results should be used by authorities to work out a guide for pedestrians to participate safely in traffic.

<h2>Data</h2>

We will use the data set recommended for this course (<b>Data Collisions - All years</b> for Seattle 2004 to now). As a first step we create a dataframe with the raw data, analyzing the given properties.

In [27]:
import pandas as pd
import numpy as np
import wget
wget.download('https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv')
raw_df = pd.read_csv('Data-Collisions.csv')
raw_df.head()

100% [........................................................................] 73917638 / 73917638

C:\Users\js423\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [28]:
raw_df.describe(include='all')

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
count,194673.000000,189339.000000,189339.000000,194673.000000,194673.000000,194673.000000,194673,194673,192747,65070.000000,...,189661,189503,4667,1.149360e+05,9333,194655,189769,194673.000000,1.946730e+05,194673
unique,NaN,NaN,NaN,NaN,NaN,NaN,194670,2,3,NaN,...,9,9,1,NaN,1,115,62,NaN,NaN,2
top,NaN,NaN,NaN,NaN,NaN,NaN,1782439,Matched,Block,NaN,...,Dry,Daylight,Y,NaN,Y,32,One parked--one moving,NaN,NaN,N
freq,NaN,NaN,NaN,NaN,NaN,NaN,2,189786,126926,NaN,...,124510,116137,4667,NaN,9333,27612,44421,NaN,NaN,187457
mean,1.298901,-122.330518,47.619543,108479.364930,141091.456350,141298.811381,NaN,NaN,NaN,37558.450576,...,NaN,NaN,NaN,7.972521e+06,NaN,NaN,NaN,269.401114,9.782452e+03,NaN
std,0.457778,0.029976,0.056157,62649.722558,86634.402737,86986.542110,NaN,NaN,NaN,51745.990273,...,NaN,NaN,NaN,2.553533e+06,NaN,NaN,NaN,3315.776055,7.226926e+04,NaN
min,1.000000,-122.419091,47.495573,1.000000,1001.000000,1001.000000,NaN,NaN,NaN,23807.000000,...,NaN,NaN,NaN,1.007024e+06,NaN,NaN,NaN,0.000000,0.000000e+00,NaN
25%,1.000000,-122.348673,47.575956,54267.000000,70383.000000,70383.000000,NaN,NaN,NaN,28667.000000,...,NaN,NaN,NaN,6.040015e+06,NaN,NaN,NaN,0.000000,0.000000e+00,NaN
50%,1.000000,-122.330224,47.615369,106912.000000,123363.000000,123363.000000,NaN,NaN,NaN,29973.000000,...,NaN,NaN,NaN,8.023022e+06,NaN,NaN,NaN,0.000000,0.000000e+00,NaN
75%,2.000000,-122.311937,47.663664,162272.000000,203319.000000,203459.000000,NaN,NaN,NaN,33973.000000,...,NaN,NaN,NaN,1.015501e+07,NaN,NaN,NaN,0.000000,0.000000e+00,NaN


<h2>Data Preparation</h2>

As a first step we will remove columns that we will not use in our examinations. These are for example columns where a description is missing in the metadata and where for this reason it is unclear what they are about, also, columns that will not help in determine risks for pedestrians will be removed.
As a second step we will remove all accidents where no pedestriaqn was involved.

In [30]:
print('Existing columns:')
list(raw_df.columns)

Existing columns:


['SEVERITYCODE',
 'X',
 'Y',
 'OBJECTID',
 'INCKEY',
 'COLDETKEY',
 'REPORTNO',
 'STATUS',
 'ADDRTYPE',
 'INTKEY',
 'LOCATION',
 'EXCEPTRSNCODE',
 'EXCEPTRSNDESC',
 'SEVERITYCODE.1',
 'SEVERITYDESC',
 'COLLISIONTYPE',
 'PERSONCOUNT',
 'PEDCOUNT',
 'PEDCYLCOUNT',
 'VEHCOUNT',
 'INCDATE',
 'INCDTTM',
 'JUNCTIONTYPE',
 'SDOT_COLCODE',
 'SDOT_COLDESC',
 'INATTENTIONIND',
 'UNDERINFL',
 'WEATHER',
 'ROADCOND',
 'LIGHTCOND',
 'PEDROWNOTGRNT',
 'SDOTCOLNUM',
 'SPEEDING',
 'ST_COLCODE',
 'ST_COLDESC',
 'SEGLANEKEY',
 'CROSSWALKKEY',
 'HITPARKEDCAR']

In [90]:
columns_to_remove = [
    "OBJECTID", "SDOT_COLCODE", "REPORTNO", "STATUS", "OBJECTID", "INCKEY", "COLDETKEY", "INTKEY", "SEVERITYCODE.1", "SEVERITYDESC", "INCDATE", 
    "SDOT_COLDESC", "ADDRTYPE", "CROSSWALKKEY", "JUNCTIONTYPE", "SDOTCOLNUM", "ST_COLDESC", "SEGLANEKEY", "EXCEPTRSNCODE", "HITPARKEDCAR", "VEHCOUNT", "PEDCYLCOUNT", "INATTENTIONIND", "UNDERINFL", "EXCEPTRSNDESC"
]
df = raw_df.drop(columns_to_remove, axis=1)

The next step would be to have a deeper look onto the values, removing statistical outliers and set meaningful defaults on missing values.
This step is done iteratively.

In [91]:
# create column ACCDATE (without date information)
# Only use date
df['ACCTIME'] = pd.to_datetime(df['INCDTTM']).dt.time
df['ACCDATE'] = pd.to_datetime(df['INCDTTM']).dt.date

# Drop additional columns that are not used
df = df.drop(['INCDTTM'], axis=1)


In [100]:
# Use Unknown as default for missing data
df['COLLISIONTYPE'].fillna('Unknown', inplace=True)
df['WEATHER'].fillna('Unknown', inplace=True)
df['ROADCOND'].fillna('Unknown', inplace=True)
df['LIGHTCOND'].fillna('Unknown', inplace=True)

# Use No as default value
df['PEDROWNOTGRNT'].fillna('N', inplace=True)
df['SPEEDING'].fillna('N', inplace=True)

for column in df:
    print('Attribute values count: ')
    print(df[column].value_counts(dropna=False))
    print('\n\n')

Attribute values count: 
2    5881
1     670
Name: SEVERITYCODE, dtype: int64



Attribute values count: 
-122.332451    24
-122.320780    23
-122.320873    23
-122.337793    22
-122.317638    21
               ..
-122.310259     1
-122.374896     1
-122.333235     1
-122.296862     1
-122.376141     1
Name: X, Length: 3064, dtype: int64



Attribute values count: 
47.607266    24
47.614076    23
47.619908    23
47.609750    22
47.661299    21
             ..
47.668521     1
47.578153     1
47.724704     1
47.551572     1
47.617852     1
Name: Y, Length: 3079, dtype: int64



Attribute values count: 
5TH AVE AND SPRING ST                                 24
BROADWAY AND E PIKE ST                                23
BROADWAY E AND E OLIVE WAY                            23
3RD AVE AND PIKE ST                                   22
ROOSEVELT WAY NE AND NE 45TH ST                       21
                                                      ..
AURORA AVE N AND GREEN LAKE DR N                  

After defining the columns that we'll use and preparing the data by using default values and so on we remove accidents that are not of interest in our avaluation.

In [99]:
#Remove accidents where no pedestrian was involved
df.drop(df[df.COLLISIONTYPE != 'Pedestrian'].index, inplace=True)
#Remove accidents without coordinates
df = df[df['X'].notna()]

The data preparation is done, in the next section we will develop a modell to predict the conditions that pedestrians should take care about. Following questions should be answered by the modell?
<ul>
    <li>Is there a specific day or time pedestrians should be careful?</li>
    <li>How does the weather and light influence the risk for pedestrians?</li>
    <li>Are there hotspots (locations) in seattle where pedestrians should be careful?</li>
</ul>